In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import joblib
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("CREACIÓN DEL PIPELINE DE INGENIERÍA DE CARACTERÍSTICAS")
print("="*80)

# ==================================================================================
# CLASE MAPPER PERSONALIZADA
# ==================================================================================

class Mapper(BaseEstimator, TransformerMixin):
    def __init__(self, variables, mappings):
        self.variables = variables
        self.mappings = mappings
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for variable in self.variables:
            X[variable] = X[variable].map(self.mappings).fillna(0)
        return X

# ==================================================================================
# TRANSFORMADOR PARA CREAR LAGS
# ==================================================================================

class LagCreator(BaseEstimator, TransformerMixin):
    def __init__(self, lags=[1, 7, 14, 30]):
        self.lags = lags
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X = X.sort_values(['Codigo_Sucursal', 'Codigo_Producto', 'Fecha_Venta'])
        
        # Solo crear lags de Unidades_Vendidas (NO de Total)
        for lag in self.lags:
            X[f'unidades_lag_{lag}'] = X.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].shift(lag)
        
        X['unidades_rolling_7'] = X.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].transform(
            lambda x: x.rolling(window=7, min_periods=1).mean()
        )
        X['unidades_rolling_30'] = X.groupby(['Codigo_Sucursal', 'Codigo_Producto'])['Unidades_Vendidas'].transform(
            lambda x: x.rolling(window=30, min_periods=1).mean()
        )
        
        return X

# ==================================================================================
# TRANSFORMADOR PARA CARACTERÍSTICAS TEMPORALES
# ==================================================================================

class TemporalFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        X['Fecha_Venta'] = pd.to_datetime(X['Fecha_Venta'], format='%d/%m/%Y', errors='coerce')
        
        X['anio'] = X['Fecha_Venta'].dt.year
        X['mes'] = X['Fecha_Venta'].dt.month
        X['dia_semana'] = X['Fecha_Venta'].dt.dayofweek
        X['dia_mes'] = X['Fecha_Venta'].dt.day
        X['trimestre'] = X['Fecha_Venta'].dt.quarter
        
        X['temporada'] = X['mes'].apply(lambda m: 
            'Invierno' if m in [12, 1, 2] else
            'Primavera' if m in [3, 4, 5] else
            'Verano' if m in [6, 7, 8] else 'Otonio'
        )
        
        return X

# ==================================================================================
# TRANSFORMADOR PARA CODIFICACIÓN DE CATEGORÍAS
# ==================================================================================

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}
    
    def fit(self, X, y=None):
        X = X.copy()
        for col in ['Codigo_Sucursal', 'Codigo_Producto', 'temporada']:
            if col in X.columns:
                self.encoders[col] = LabelEncoder()
                self.encoders[col].fit(X[col].astype(str))
        return self
    
    def transform(self, X):
        X = X.copy()
        for col, encoder in self.encoders.items():
            if col in X.columns:
                X[f'{col}_encoded'] = encoder.transform(X[col].astype(str))
        return X

# ==================================================================================
# TRANSFORMADOR PARA TRATAMIENTO DE OUTLIERS
# ==================================================================================

class OutlierTreatment(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.limits = {}
    
    def fit(self, X, y=None):
        X = X.copy()
        for col in self.columns:
            if col in X.columns:
                Q1 = X[col].quantile(0.25)
                Q3 = X[col].quantile(0.75)
                IQR = Q3 - Q1
                self.limits[col] = {
                    'lower': Q1 - 1.5 * IQR,
                    'upper': Q3 + 1.5 * IQR
                }
        return self
    
    def transform(self, X):
        X = X.copy()
        for col, limits in self.limits.items():
            if col in X.columns:
                X[f'{col}_tratado'] = X[col].clip(limits['lower'], limits['upper'])
        return X

# ==================================================================================
# TRANSFORMADOR PARA TRANSFORMACIONES LOGARÍTMICAS
# ==================================================================================

class LogTransformation(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            if col in X.columns:
                X[f'{col}_log'] = np.log1p(X[col])
        return X

# ==================================================================================
# TRANSFORMADOR PARA SELECCIÓN DE CARACTERÍSTICAS
# ==================================================================================

class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = features
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        available_features = [f for f in self.features if f in X.columns]
        return X[available_features]

# ==================================================================================
# DEFINICIÓN DE VARIABLES Y CONFIGURACIÓN
# ==================================================================================

NUMERICAL_VARS_IMPUTE = ['unidades_lag_1', 'unidades_lag_7', 'unidades_lag_14', 
                         'unidades_lag_30']  # ← Quité total_lag_*

OUTLIER_VARS = ['Unidades_Vendidas']  # ← Quité 'Total'

LOG_TRANSFORM_VARS = ['Unidades_Vendidas_tratado']  # ← Quité 'Total_tratado'

FEATURES_TO_SCALE = ['unidades_lag_1', 'unidades_lag_7', 'unidades_lag_14',
                     'unidades_rolling_7', 'unidades_rolling_30', 'mes', 
                     'dia_mes', 'dia_semana']

FEATURES_FOR_MODEL = ['Codigo_Sucursal_encoded', 'Codigo_Producto_encoded',
                      'anio', 'mes', 'dia_semana', 'trimestre', 'temporada_encoded',
                      'unidades_lag_1', 'unidades_lag_7', 'unidades_lag_14', 
                      'unidades_lag_30', 'unidades_rolling_7', 'unidades_rolling_30',
                      'Unidades_Vendidas_tratado_log']  # ← Quité Total_tratado_log

from sklearn.compose import ColumnTransformer

# ==================================================================================
# CONSTRUCCIÓN DEL PIPELINE 
# ==================================================================================

# Pipeline de imputación solo para columnas numéricas
numerical_imputer = Pipeline([
    ('imputer', SimpleImputer(strategy='median'))
])

# Pipeline de escalado solo para columnas numéricas finales
scaler = Pipeline([
    ('scaler', StandardScaler())
])

# ColumnTransformer que aplica transformaciones específicas a subconjuntos
preprocessor = ColumnTransformer(transformers=[
    ('num_imputer', numerical_imputer, NUMERICAL_VARS_IMPUTE),
    ('scaler', scaler, FEATURES_TO_SCALE)
], remainder='passthrough')  # deja pasar las demás columnas

# Pipeline principal
ventas_preprocessing_pipeline = Pipeline([
    ('temporal_features', TemporalFeatures()),
    ('lag_creator', LagCreator(lags=[1, 7, 14, 30])),
    ('categorical_encoder', CategoricalEncoder()),
    ('outlier_treatment', OutlierTreatment(columns=OUTLIER_VARS)),
    ('log_transformation', LogTransformation(columns=LOG_TRANSFORM_VARS)),
    ('preprocessor', preprocessor)
])

print("\nPipeline de preprocesamiento creado exitosamente")
print(f"Pasos del pipeline: {len(ventas_preprocessing_pipeline.steps)}")

# ==================================================================================
# CARGA Y PREPARACIÓN DE DATOS PARA AJUSTE
# ==================================================================================

print("\n" + "="*80)
print("CARGANDO DATOS PARA AJUSTAR EL PIPELINE")
print("="*80)

df = pd.read_csv(r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\data\raw\ventas.csv')
print(f"Datos cargados: {len(df):,} registros")

# Ajustar el pipeline con los datos
X_sample = df.drop(['Total'], axis=1) if 'Total' in df.columns else df
ventas_preprocessing_pipeline.fit(X_sample)

print("\nPipeline ajustado exitosamente")

# ==================================================================================
# EXPORTAR PIPELINE
# ==================================================================================

pipeline_path = 'models'
joblib.dump(ventas_preprocessing_pipeline, pipeline_path)

print("\n" + "="*80)
print("PIPELINE GUARDADO")
print("="*80)
print(f"Ubicación: {pipeline_path}")
print("\nEl pipeline está listo para ser usado en entrenamiento e inferencia")


CREACIÓN DEL PIPELINE DE INGENIERÍA DE CARACTERÍSTICAS

Pipeline de preprocesamiento creado exitosamente
Pasos del pipeline: 6

CARGANDO DATOS PARA AJUSTAR EL PIPELINE
Datos cargados: 73,153 registros

Pipeline ajustado exitosamente

PIPELINE GUARDADO
Ubicación: models

El pipeline está listo para ser usado en entrenamiento e inferencia
